### This code generates more diverse augmenteed images than augment_1 (used for multimodal).

In [2]:
import os 
import cv2
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import ToPILImage
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import torch.optim as optim
from torchvision import transforms
from sklearn.model_selection import train_test_split
import shutil

from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import random

from tqdm import tqdm

from torchvision.transforms import RandomRotation, ColorJitter, RandomHorizontalFlip, RandomCrop
from torchvision.transforms import RandomAffine

from torch.optim.lr_scheduler import CyclicLR

In [ ]:
from PIL import Image, ImageDraw
import os

def mask_image(image, rect_coords):
    # Create a black mask
    mask = Image.new('L', image.size, 0)  
    draw = ImageDraw.Draw(mask)
    
    # Draw white rectangles for the specified areas
    for rect in rect_coords:
        x1, y1 = min(rect[0], rect[2]), min(rect[1], rect[3])
        x2, y2 = max(rect[0], rect[2]), max(rect[1], rect[3])
        draw.rectangle([x1, y1, x2, y2], fill=255)

    # Apply the mask to the image
    masked_image = Image.new('RGB', image.size, (0, 0, 0))  
    masked_image.paste(image, mask=mask)  
    return masked_image

def process_and_save_frames(input_dir, output_dir, rect_coords):
    os.makedirs(output_dir, exist_ok=True)
    
    # for each match folder
    for match_id in os.listdir(input_dir):
        match_folder = os.path.join(input_dir, match_id)
        if not os.path.isdir(match_folder):
            continue
        
        output_match_folder = os.path.join(output_dir, match_id)
        os.makedirs(output_match_folder, exist_ok=True)

        # apply mask to each frame
        for frame in os.listdir(match_folder):
            if not frame.endswith('.jpg'):
                continue

            frame_path = os.path.join(match_folder, frame)
            image = Image.open(frame_path).convert("RGB")
            masked_image = mask_image(image, rect_coords)

            # save
            output_path = os.path.join(output_match_folder, frame)
            masked_image.save(output_path)

    print(f"Masked images saved to {output_dir}")

In [ ]:
input_dir = './data/frames'
output_dir = './data/masked_frames'
rect_coords = [
    (675, 10, 1242, 85),
    (1755, 5, 1916, 72),
    (292, 95, 2, 5),
    (1630, 793, 1915, 1074),
    (597, 846, 1335, 1077),
    (1828, 126, 1918, 648),
    (83, 132, 1, 655),
]

process_and_save_frames(input_dir, output_dir, rect_coords)

Masked images saved to ./data/masked_frames


In [ ]:
def augment_image_multiple(image, num_augmentations=10):
    # apply augmentation
    augmentations = transforms.Compose([
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomCrop(size=(image.size[1] - 50, image.size[0] - 50)),
        transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1))
    ])
    
    augmented_images = [augmentations(image) for _ in range(num_augmentations)]
    return augmented_images



def augment_train_data(train_dir, num_augmentations=10):
    # apply augmentation only for the training data
    matches = os.listdir(train_dir)

    for match_id in tqdm(matches, desc="Processing Matches", unit="match"):
        match_folder = os.path.join(train_dir, match_id)
        if not os.path.isdir(match_folder):
            continue

        frames = [f for f in os.listdir(match_folder) if f.endswith('.jpg')]

        for frame in tqdm(frames, desc=f"Augmenting Frames in Match {match_id}", unit="frame", leave=False):
            frame_path = os.path.join(match_folder, frame)
            image = Image.open(frame_path).convert("RGB")
            augmented_images = augment_image_multiple(image, num_augmentations=num_augmentations)

            for i, augmented_image in enumerate(augmented_images):
                augmented_path = os.path.join(
                    match_folder, f"{os.path.splitext(frame)[0]}_aug_{i+1}.jpg"
                )
                augmented_image.save(augmented_path)

    print(f"Augmentation completed for {train_dir}, with {num_augmentations} augmentations per image.")

In [ ]:
# apply augmentation to the training data
train_dir = './data/masked_frame_split_by_match/train'
augment_train_data(train_dir, num_augmentations=10)

Processing Matches: 100%|██████████| 63/63 [1:01:09<00:00, 58.24s/match]

Augmentation completed for ./data/masked_frame_split_by_match/train, with 10 augmentations per image.
